In [1]:
filename = '2022-05 16 Fact 9-May al 15-May.xlsx'
year = 2022
week = 20

# Libraries

In [2]:
import pandas as pd
import pyodbc
import os
from datetime import datetime

pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.float_format = '{:,.2f}'.format

# Parameters

In [3]:
path = r'\\NASPRO.infovisiontv.com\DGI\DGI Banco Central\00 CIERRES SEMANALES\{0}\{0} {1}'
path = path.format(year, str(week).zfill(2))

# Import Data

'Grupo de artículos'

In [4]:
%%time
#df = pd.read_excel(path + '\\' + filename, sheet_name='YOSAF', dtype={'Grupo de artículos':'str', 'Tp.cambio p.contab.':'str', 'Lote':'str'})
df = pd.read_excel(path + '\\' + filename, dtype={'Grupo de artículos':'str', 'Tp.cambio p.contab.':'str', 'Lote':'str'})

Wall time: 16.9 s


df[['Grupo de artículos', 'Tp.cambio p.contab.', 'Lote']].dtypes

In [5]:
df.shape

(14022, 47)

In [6]:
df.tail(10)

,Doc.fact.,Posición,Documento de ventas,Posición.1,Referencia cliente,Material,Cd.Barra,Denomin.posición,Solicitante,Nombre,...,Nombre empresa,Organización ventas,Centro,Almacén,Lote,FeCaduc/FePreferCons,Motivo pedido,Descripción Motivo Pedido,Valor neto.1,Moneda del documento
14012,"196,269,544.00",260.00,"15,320,229.00",260.00,TRC-72902,"8,100,003,234.00","650,240,013,850.00",TEATRICAL CRM SLD LANOLINA AZUL 52G 0917,"1,100,885.00",DIFARMER SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,2084005205,2024-03-30,NaN,NaN,"1,334.16",MXN
14013,"196,269,544.00",270.00,"15,320,229.00",270.00,TRC-72902,"8,100,000,888.00","650,240,010,507.00",ULTRA BENGUE AZUL 35 G,"1,100,885.00",DIFARMER SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,M22056,2024-02-28,NaN,NaN,"3,840.00",MXN
14014,"196,269,544.00",280.00,"15,320,229.00",280.00,TRC-72902,"8,100,000,881.00","650,240,006,647.00",ULTRA BENGUE 35GR NUEVO ARTE,"1,100,885.00",DIFARMER SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,M21362,2023-11-30,NaN,NaN,"7,680.00",MXN
14015,"196,269,544.00",300.00,"15,320,229.00",300.00,TRC-72902,"8,100,003,489.00","650,240,017,117.00",XL3 ANTIGRIPAL 8TAB 0518,"1,100,885.00",DIFARMER SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,1389,2023-11-30,NaN,NaN,"6,692.00",MXN
14016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"172,571,075.93",MXN
14017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"1,760,945.41",USD
14018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
14019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
14020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
14021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


# Data Cleaning

In [7]:
df = df[:df.shape[0]-6]

In [8]:
df.isnull().sum()[df.isnull().sum() > 0]

Referencia cliente             257
Cd.Barra                      1623
RFC                           3522
Nº cuenta anterior            1456
Destinatario mcía.             257
Nombre.1                       257
Descripción CeBe               257
Almacén                        312
Lote                           446
FeCaduc/FePreferCons           446
Motivo pedido                13018
Descripción Motivo Pedido    13018
dtype: int64

In [9]:
new_cols = [
     'Factura',
     'Posicion',
     'DocumentoVentas',
     'PosicionDV',
     'NumeroPedCliente',
     'Material',
     'CodigoBarras',
     'Denominacion',
     'Solicitante',
     'Nombre',
     'RFC',
     'CNAC',
     'GrupoCuentas',
     'GrupoTesoreria',
     'DestinatarioMcia',
     'NombreDestinatario',
     'CantidadFacturada',
     'UnidadMedida',
     'CtdFactUMB',
     'UnidadMedidaBase',
     'PrecioUnitario',
     'ImpuestoBase',
     'ImpuestoDescuento',
     'ValorNeto',
     'ImporteImpuesto',
     'ImpuestoIEPS',
     'ImpuestoTotal',
     'CentroBeneficio',
     'DescripcionCebe',
     'TipoUnidades',
     'FechaFactura',
     'GrupoArticulos',
     'DenominacionArticulos',
     'ClaseFactura',
     'DenominacionTipoDocto',
     'CambioContable',
     'Sociedad',
     'Nombreempresa',
     'OrganizacionVentas',
     'Centro',
     'Almacen',
     'Lote',
     'FechaCaducidad',
     'MotivoPedido',
     'DescMotivoPedido',
     'ValorNetoAbs',
     'Moneda'
]

In [10]:
df.columns = new_cols

In [11]:
cols_nulls = ['RFC', 'CNAC', 'Almacen', 'Lote', 'MotivoPedido', 'DescMotivoPedido']
for col in cols_nulls: 
    df[col].fillna('     ', inplace=True)

In [12]:
df.isnull().sum()[df.isnull().sum() > 0]

NumeroPedCliente       257
CodigoBarras          1623
DestinatarioMcia       257
NombreDestinatario     257
DescripcionCebe        257
FechaCaducidad         446
dtype: int64

In [13]:
cols_int = [
    'Factura',
    'Posicion',
    'DocumentoVentas',
    'PosicionDV',
    'Material',
    'Solicitante',
    'CtdFactUMB',
    'ImpuestoDescuento',
    'ImpuestoIEPS',
]

In [14]:
for col in cols_int:
    df[col] = df[col].map(int)

In [15]:
df[['FechaCaducidad', 'FechaCaducidad']].dtypes

FechaCaducidad    datetime64[ns]
FechaCaducidad    datetime64[ns]
dtype: object

# Analysis

In [16]:
print('FechaFactura desde:', df['FechaFactura'].min())
print('FechaFactura hasta:', df['FechaFactura'].max())

FechaFactura desde: 2022-05-09 00:00:00
FechaFactura hasta: 2022-05-15 00:00:00


In [17]:
print('FechaCaducidad desde:', df['FechaCaducidad'].min())
print('FechaCaducidad hasta:', df['FechaCaducidad'].max())

FechaCaducidad desde: 2020-11-30 00:00:00
FechaCaducidad hasta: 2031-06-14 00:00:00


# Export Data

In [18]:
df.to_excel(path + '\\Cargar ' + filename.split('.')[0] + '.xlsx',
            index=False,
            sheet_name='carga')

In [19]:
df.shape

(14016, 47)

# Validations

In [20]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Trusted_Connection=yes;')

In [21]:
qtime = '''SELECT DISTINCT 
                TmpFecha,
                TmpSemanaAnioGenomma AS SemanaGL,
                TmpAnioSemanaGenomma AS AnioGL
            FROM Gnm_DWH.dbo.Dim_Tiempo'''

In [22]:
dtime = pd.read_sql(qtime, conn)

In [23]:
df = df.merge(dtime, left_on='FechaFactura', right_on='TmpFecha', how='left')

In [24]:
df.pivot_table(
    index='SemanaGL', 
    values=['ValorNeto'],
    aggfunc='sum'
)

,ValorNeto
SemanaGL,
20,"163,725,401.95"


In [25]:
df.pivot_table(
    index='SemanaGL', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
SemanaGL,
20,14016
All,14016


In [26]:
df.pivot_table(
    index='FechaFactura', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
FechaFactura,
2022-05-09 00:00:00,2597
2022-05-10 00:00:00,2388
2022-05-11 00:00:00,2660
2022-05-12 00:00:00,1674
2022-05-13 00:00:00,1366
2022-05-14 00:00:00,1484
2022-05-15 00:00:00,1847
All,14016


In [27]:
df.pivot_table(
    index='MotivoPedido', 
    values='ValorNeto', 
    aggfunc='sum'
)

,ValorNeto
MotivoPedido,
,"175,960,434.45"
M03,"-927,632.78"
M05,"-38,959.40"
M06,"-28,695.65"
M07,"-438,454.75"
M08,"-2,205,495.43"
M09,"-466,265.81"
M11,"-96,242.40"
M13,"-280,553.95"
